In [ ]:
# Loading necessary modlules
import os 
import os.path as osp
import numpy as np
import matplotlib.pyplot as pl
pl.rcParams['axes.labelsize'] = 16
pl.rcParams['axes.titlesize'] = 16
import itertools

import pandas as pd
import seaborn as sns
import timeit
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
# Scores
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import f1_score as f1
from sklearn.metrics import recall_score as recall
from sklearn.metrics import precision_score as precision
from sklearn.metrics import classification_report, make_scorer

from imblearn.under_sampling import RandomUnderSampler
# ML models  
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from pprint import pprint
pl.style.use('seaborn-ticks')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [ ]:
# The train data
X_data = pd.read_csv('X_train_new.csv')
y_data = pd.read_csv('y_train_new.csv')

# The Unseen test data
X_test = pd.read_csv('X_test_new.csv')
y_test = pd.read_csv('y_test_new.csv')

# y = y_data['labels']
y_tr = np.array(y_data).ravel()

In [ ]:
y_tr

In [ ]:
X_train, X_vald, y_train, y_vald = train_test_split(X_data, y_data, stratify = y_data, test_size=0.2, random_state=42)

In [ ]:
## random forest (RF)
# The Random Hyper parameter Grid

# number of trees in the forest
n_estimators = [50, 100, 150]

# Number of feature to consider at every split
max_features = [2, 3]

# Maximum number of levels in tree
max_depth = [5, 10]

# Minimum number of samples required to split a node
min_samples_split = [2, 5]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3]

# Method of selecting samples for training each tree
bootstrap = [True, False]


# Create the random grid
rf_par = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth}
               # 'min_samples_split': min_samples_split,
               # 'min_samples_leaf': min_samples_leaf,
               # 'bootstrap': bootstrap}



rf_model= RandomForestClassifier(random_state=1)
rf_par = dict(n_estimators=n_estimators)

In [ ]:
X_vald_test = pd.concat([X_vald, X_test], sort=False)

# y_vald_df = pd.DataFrame(y_vald, columns = ['labels'])
y_vald_test = pd.concat([y_vald, y_test], sort=False)

# X_vald_test['label'] =  y_vald_test
# X_vald_test.reset_index(drop=True)
# y_vald_test.reset_index(drop=True)


In [ ]:
# CLASSIFIER AND METRICS ALGORITHMS

# Source Classification
start_time = timeit.default_timer()

rf_model.fit(X_train, np.array(y_train).ravel())  

y_pred = rf_model.predict(X_vald_test)
y_pred_vald = rf_model.predict(X_vald)
y_pred_test = rf_model.predict(X_test)



elapsed = timeit.default_timer() - start_time

proba = rf_model.predict_proba(X_test)



In [ ]:
def confusion_matrix(cm, classes, 
                        name = '',
                        normalize=False,
                        title='Confusion Matrix',
                        cmap=pl.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    pl.imshow(cm, interpolation='nearest', cmap=cmap)
    pl.title(title, fontsize=26)
    pl.colorbar()
    tick_marks = np.arange(len(classes))
    pl.xticks(tick_marks, classes, fontsize=20,rotation=45)
    pl.yticks(tick_marks, classes, fontsize=20)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        pl.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=28)

    pl.tight_layout()
    pl.ylabel('True Label', fontsize=20)
    pl.xlabel('Predicted Label', fontsize=20)
    pl.savefig(name)
    pl.show()# Feature importance for the experiment

In [ ]:
# Compute confusion matrix for kNN classifier
cm_test = metrics.confusion_matrix(y_test, y_pred_test)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
pl.figure(figsize=(11,11))
confusion_matrix(cm_test,classes=['AGN','SFG'], name = 'cm_fluxes', normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
# Compute confusion matrix for kNN classifier
cm = metrics.confusion_matrix(y_vald_test, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
pl.figure(figsize=(11,11))
confusion_matrix(cm,classes=['AGN','SFG'], name = 'cm_fluxes', normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
# Compute confusion matrix for kNN classifier
cm_noxray = metrics.confusion_matrix(y_vald, y_pred_vald)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
pl.figure(figsize=(11,11))
confusion_matrix(cm_noxray,classes=['AGN','SFG'], name = 'cm_fluxes', normalize=True,
                      title='Normalized confusion matrix')

---

## Experiments to produce F1, Precision and Recall

In [ ]:
X_data = pd.read_csv('final-train-test/X_train.csv')
y_data = pd.read_csv('final-train-test/y_train.csv')

# The Unseen test data
X_test = pd.read_csv('final-train-test/X_test.csv')
y_test = pd.read_csv('final-train-test/y_test.csv')

referece_test = pd.read_csv('final-train-test/original_test_df.csv')


y = y_data['labels']

# drop catid
X_data = X_data.drop(["CATID"], axis='columns')
X_test = X_test.drop(["CATID"], axis='columns')

In [ ]:
# # The train data
# X_data_m = pd.read_csv('X_train.csv')
# y_data_m = pd.read_csv('y_train.csv')

# # The Unseen test data
# X_test_m = pd.read_csv('X_test.csv')
# y_test_m = pd.read_csv('y_test.csv')


# # With CatID
# # X_data_tb = pd.read_csv('X_train_table.csv')
# # y_data_tb = pd.read_csv('y_train_table.csv')
# # X_tb_train = X_data_tb.drop(['index','CATID'], axis = 1)
# # # X_tb_train = X_tb_train.dropna()

# # X_test_tb = pd.read_csv('X_test_table.csv')
# # y_test_tb = pd.read_csv('y_test_table.csv')
# # X_tb_test = X_test_tb.drop(['index','CATID'], axis = 1)
# # X_tb_test = X_tb_test.dropna()

In [ ]:
# data = pd.read_csv('raw_data.csv')
# data = data.dropna()
# # data = data.drop(['index','CATID'], axis = 1)
# data = data.drop(['CATID'], axis = 1)


# labels = data['class_labels']
# X_tb = data.drop(['class_labels'], axis = 1)

# # encoding target class
# y_tb, clas_tb = pd.factorize(labels) #getting the class 0 = agn, 1 =notagn, 2 = no class
# y_target_tb = pd.DataFrame(y_tb, columns = ['labels'])

In [ ]:
# data

In [ ]:
X_train_tb, X_test_tb, y_train_tb, y_test_tb = train_test_split(X_data, y_data, stratify = y_data, test_size=0.25, random_state=42)

In [ ]:
# print(X_train_tb.isna().sum().sum())
# print(X_test_tb.isna().sum().sum())


In [ ]:
# X_train_tb_noindex = X_train_tb.drop(['index'], axis = 1)

# X_test_tb_noindex = X_test_tb.drop(['index'], axis = 1)


In [ ]:
# def get_score(model, X_train, X_test, y_train, y_test):

#     y_tr = np.array(y_data).ravel()
    
#     # CLASSIFIER AND METRICS ALGORITHMS
#     model.fit(X_train, np.array(y_tr).ravel())  
    
#     # Predict
#     y_pred = rf_model.predict(X_test)

#     #scores
#     f1_res = f1(y_tr, y_pred)
#     recall_res = recall(y_tr, y_pred)
#     precision_res = prcision(y_tr, y_pred)

#     #print the results


In [ ]:
### Original dataset

X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

# The Unseen test data
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')


In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import f1_score as f1, recall_score as recall, precision_score as precision
# import numpy as np

def get_score(models, X_train, X_test, y_train, y_test, cv=5, save_to_file=False, filename='model_scores.txt', class_to_predict=1):
    """
    Evaluate multiple models using cross-validation and print/save F1, Recall, and Precision scores.
    Returns the trained models along with evaluation results.
    
    Parameters:
    - models: List of models to evaluate.
    - X_train, X_test, y_train, y_test: Training and testing datasets.
    - cv: Number of folds for cross-validation (e.g., 3, 5, 10).
    - save_to_file: Boolean, whether to save the results to a file.
    - filename: Name of the file to save the results.
    
    Returns:
    - results: List of dictionaries containing evaluation results and trained models.
    """
    
    y_tr = np.array(y_train).ravel()
    y_te = np.array(y_test).ravel()
    
    results = []
    
    for model in models:
        # Perform cross-validation
        f1_scores = cross_val_score(model, X_train, y_tr, cv=cv, scoring='f1')
        recall_scores = cross_val_score(model, X_train, y_tr, cv=cv, scoring='recall')
        precision_scores = cross_val_score(model, X_train, y_tr, cv=cv, scoring='precision')
        
        # Compute mean and standard deviation of scores
        f1_mean, f1_std = f1_scores.mean(), f1_scores.std()
        recall_mean, recall_std = recall_scores.mean(), recall_scores.std()
        precision_mean, precision_std = precision_scores.mean(), precision_scores.std()
        
        # Train the model on the full training set
        model.fit(X_train, y_tr)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Compute test scores
        f1_test = f1(y_te, y_pred, zero_division=0)
        recall_test = recall(y_te, y_pred, zero_division=0)
        precision_test = precision(y_te, y_pred, zero_division=0)
        
        # Store results and the trained model
        results.append({
            'model': model.__class__.__name__,
            'trained_model': model,  # Store the trained model
            'f1_mean': f1_mean,
            'f1_std': f1_std,
            'recall_mean': recall_mean,
            'recall_std': recall_std,
            'precision_mean': precision_mean,
            'precision_std': precision_std,
            'f1_test': f1_test,
            'recall_test': recall_test,
            'precision_test': precision_test,
            'y_pred': y_pred
        })
        
        # Print results
        print(f"Model: {model.__class__.__name__}")
        print(f"CV F1 Score: {f1_mean:.4f} (±{f1_std:.4f})")
        print(f"CV Recall: {recall_mean:.4f} (±{recall_std:.4f})")
        print(f"CV Precision: {precision_mean:.4f} (±{precision_std:.4f})")
        print(f"Test F1 Score: {f1_test:.4f}")
        print(f"Test Recall: {recall_test:.4f}")
        print(f"Test Precision: {precision_test:.4f}")
        print("-" * 40)
    
    # Save results to file if requested
    if save_to_file:
        with open(filename, 'w') as f:
            for res in results:
                f.write(f"Model: {res['model']}\n")
                f.write(f"CV F1 Score: {res['f1_mean']:.4f} (±{res['f1_std']:.4f})\n")
                f.write(f"CV Recall: {res['recall_mean']:.4f} (±{res['recall_std']:.4f})\n")
                f.write(f"CV Precision: {res['precision_mean']:.4f} (±{res['precision_std']:.4f})\n")
                f.write(f"Test F1 Score: {res['f1_test']:.4f}\n")
                f.write(f"Test Recall: {res['recall_test']:.4f}\n")
                f.write(f"Test Precision: {res['precision_test']:.4f}\n")
                f.write("-" * 40 + "\n")
        print(f"Results saved to {filename}")

    return results

# Example usage:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# models = [RandomForestClassifier(), LogisticRegression()]
# results = get_score(models, X_train, X_test, y_train, y_test, cv=5, save_to_file=True)

# Accessing trained models:
# trained_rf = results[0]['trained_model']  # Trained RandomForestClassifier
# trained_lr = results[1]['trained_model']  # Trained LogisticRegression

In [ ]:
# ML Models
rf_model2 = RandomForestClassifier(random_state=1)
svm_model = SVC(kernel='linear')
knn_model = KNeighborsClassifier()
lr_model = LogisticRegression()
xgb_model = xgboost.XGBClassifier(use_label_encoder=False, eval_metric='rmse', n_jobs=-1 )

models = [lr_model, svm_model, knn_model, rf_model2, xgb_model]


### WRITE UP

In [ ]:
# spliting data 20 % test / 80 % train
# X_train_splt02, X_test_splt02, y_train_splt02, y_test_splt02 = train_test_split(X_train_tb_noindex, y_train_tb, stratify = y_train_tb, test_size=0.8, random_state=42)
X_train_splt02, X_test_splt02, y_train_splt02, y_test_splt02 = train_test_split(X_train, y_train, stratify = y_train, test_size=0.8, random_state=42)

results = get_score(models, X_train_splt02, X_test_splt02, y_train_splt02, y_test_splt02, save_to_file=True, filename='ml_review_results/model_scores_02.txt', class_to_predict = 1)


In [ ]:
# models = [lr_model, svm_model, knn_model, rf_model2, xgb_model]

# Access trained models
trained_lr = results[0]['trained_model']  # Trained RandomForestClassifier
trained_svm = results[1]['trained_model']  # Trained LogisticRegression
trained_knn = results[2]['trained_model']  # Trained LogisticRegression
trained_rf = results[3]['trained_model']  # Trained LogisticRegression
trained_xgb = results[4]['trained_model']  # Trained LogisticRegression

# Use the trained models for further predictions
new_predictions_lr = trained_lr.predict(X_test_tb_noindex)
new_predictions_svm = trained_svm.predict(X_test_tb_noindex)
new_predictions_knn = trained_knn.predict(X_test_tb_noindex)
new_predictions_rf = trained_rf.predict(X_test_tb_noindex)
new_predictions_xgb = trained_xgb.predict(X_test_tb_noindex)


In [ ]:
len(new_predictions_lr)

In [ ]:
len(y_test_tb)

In [ ]:
def arrays_to_df(predicted, column_names):
    """
    Convert a list of arrays into a pandas DataFrame.

    Parameters:
    arrays (list of array-like): List of 1D arrays or lists, each representing a column.
    columns (list of str): List of column names.

    Returns:
    pd.DataFrame: A DataFrame constructed from the given arrays.
    """
    if len(predicted) != len(column_names):
        raise ValueError("Number of arrays must match the number of column names")

    # Transpose the list of arrays to match DataFrame structure
    data = np.column_stack(predicted)  # Ensures alignment of data
    return pd.DataFrame(data, columns=column_names)

In [ ]:
predicted_out_put = [new_predictions_lr, new_predictions_svm, new_predictions_knn, new_predictions_rf, new_predictions_xgb]
col_names = ['lr', 'svm', 'knn', 'rf', 'xgb']

outcomes = arrays_to_df(predicted_out_put, col_names)

In [ ]:
outcomes['index'] = np.array(X_test_tb['index'])

In [ ]:
full_test_df = pd.merge(X_test_tb, outcomes, on='index', how='inner')

In [ ]:
full_test_df['true_label'] = np.array(y_test_tb)

In [ ]:
catalog = pd.read_csv('raw_data.csv')


In [ ]:
full_test_df_withID = pd.merge(catalog, full_test_df, on='index', how='inner')

In [ ]:
import astropy.io.fits as fits
from astropy.table import Table

file = osp.join("COSMOSXMATCH+classes_040422_withphotometry.fits")

hdu_list = fits.open(file)

evt_data = Table(hdu_list[1].data)

In [ ]:
mightee_data = evt_data.to_pandas()

In [ ]:
full_test_df_withID_all_class = pd.merge(full_test_df_withID, mightee_data, on='CATID', how='inner')

In [ ]:
full_test_df_withID_all_class

In [ ]:
full_test_df_withID_all_class.to_csv('classified_data_08.csv', index = False, header=True)

In [ ]:
catalog_agn_sfg_xray = full_test_df_withID_all_class[full_test_df_withID_all_class['XAGN'] == True]
catalog_agn_sfg_vlbi = full_test_df_withID_all_class[full_test_df_withID_all_class['VLBAAGN'] == True]

In [ ]:
print(catalog_agn_sfg_xray.columns)

In [ ]:
col_names_xray = ['CATID', 'XAGN', 'lr', 'svm', 'knn', 'rf', 'xgb', 'true_label']
col_names_vlbi = ['CATID', 'VLBAAGN', 'lr', 'svm', 'knn', 'rf', 'xgb', 'true_label']

catalog_agn_sfg_xray = catalog_agn_sfg_xray[col_names_xray]
catalog_agn_sfg_vlbi = catalog_agn_sfg_vlbi[col_names_vlbi]

In [ ]:
catalog_agn_sfg_xray.to_csv('ml_review_results/catalog_agn_sfg_xray_02.csv', index = False, header=True)

In [ ]:
catalog_agn_sfg_vlbi.to_csv('ml_review_results/catalog_agn_sfg_vlbi_02.csv', index = False, header=True)


In [ ]:
## DONE FISRT PART

---
---

In [ ]:
# # spliting data 80 % test / 20 % train
# X_train_splt08, X_test_splt08, y_train_splt08, y_test_splt08 = train_test_split(X_data_m, y_data_m, stratify = y_data_m, test_size=0.8, random_state=42)

# all_res_splt = get_score(models, X_train_splt08, X_test_splt08, y_train_splt08, y_test_splt08, save_to_file=False, filename='model_scores.txt', class_to_predict = 1)


### Full Data

In [ ]:
# all_results = get_score(models, X_data_m, X_test_m, y_data_m, y_test_m, save_to_file=False, filename='model_scores.txt', class_to_predict = 1)

In [ ]:
# noxrayvlbi_data = get_score(models, X_train, X_vald, y_train, y_vald, save_to_file=False, filename='model_scores.txt', class_to_predict = 1)


### Undersampling

In [ ]:
# # Resampling the SFGs to equal the galaxies
# under_sampler = RandomUnderSampler(random_state=42)


# X_res, y_res = under_sampler.fit_resample(X_train, y_train)

### ONLY VLBI

In [ ]:
# # The Unseen test data
# X_vlba = pd.read_csv('X_vlba.csv')
# y_vlba = pd.read_csv('y_vlba.csv')

# # y = y_data['labels']
# # y_tr = np.array(y_data).ravel()
# y_vlb, clas_vlb = pd.factorize(y_vlba["class_labels"]) 

In [ ]:
# y_vlb

In [ ]:
# results_vlba = get_score(models, X_res, X_vlba, y_res, y_vlb, save_to_file=False, filename='model_scores_xray.txt', class_to_predict = 1)

In [ ]:
# results_vlba

### ONLY XRAY

In [ ]:
# # The Unseen test data
# X_xray = pd.read_csv('X_xray.csv')
# y_xray = pd.read_csv('y_xray.csv')

# # y = y_data['labels']
# # y_tr = np.array(y_data).ravel()
# y_ray, clas_ray = pd.factorize(y_xray["class_labels"]) 

In [ ]:
# result_xray = get_score(models, X_res, X_xray, y_res, y_ray, save_to_file=False, filename='model_scores_vlba.txt', class_to_predict = 1)

In [ ]:
# OLD FUNCTIONS

# def get_score(models, X_train, X_test, y_train, y_test, cv=5, save_to_file=False, filename='model_scores.txt', class_to_predict=1):
#     """
#     Evaluate multiple models using cross-validation and print/save F1, Recall, and Precision scores.
    
#     Parameters:
#     - models: List of models to evaluate.
#     - X_train, X_test, y_train, y_test: Training and testing datasets.
#     - cv: Number of folds for cross-validation (e.g., 3, 5, 10).
#     - save_to_file: Boolean, whether to save the results to a file.
#     - filename: Name of the file to save the results.
#     """
#     y_tr = np.array(y_train).ravel()
#     y_te = np.array(y_test).ravel()
    
#     results = []
    
#     for model in models:
#         # Perform cross-validation
#         f1_scores = cross_val_score(model, X_train, y_tr, cv=cv, scoring='f1')
#         recall_scores = cross_val_score(model, X_train, y_tr, cv=cv, scoring='recall')
#         precision_scores = cross_val_score(model, X_train, y_tr, cv=cv, scoring='precision')
        
#         # Compute mean and standard deviation of scores
#         f1_mean, f1_std = f1_scores.mean(), f1_scores.std()
#         recall_mean, recall_std = recall_scores.mean(), recall_scores.std()
#         precision_mean, precision_std = precision_scores.mean(), precision_scores.std()
        
#         # Train the model on the full training set
#         model.fit(X_train, y_tr)
        
#         # Predict on the test set
#         y_pred = model.predict(X_test)
        
#         # Compute test scores
#         f1_test = f1(y_te, y_pred, zero_division=0)
#         recall_test = recall(y_te, y_pred, zero_division=0)
#         precision_test = precision(y_te, y_pred, zero_division=0)
        
#         # Store results
#         results.append({
#             'model': model.__class__.__name__,
#             'f1_mean': f1_mean,
#             'f1_std': f1_std,
#             'recall_mean': recall_mean,
#             'recall_std': recall_std,
#             'precision_mean': precision_mean,
#             'precision_std': precision_std,
#             'f1_test': f1_test,
#             'recall_test': recall_test,
#             'precision_test': precision_test,
#             'y_pred': y_pred
#         })
        
#         # Print results
#         print(f"Model: {model.__class__.__name__}")
#         print(f"CV F1 Score: {f1_mean:.4f} (±{f1_std:.4f})")
#         print(f"CV Recall: {recall_mean:.4f} (±{recall_std:.4f})")
#         print(f"CV Precision: {precision_mean:.4f} (±{precision_std:.4f})")
#         print(f"Test F1 Score: {f1_test:.4f}")
#         print(f"Test Recall: {recall_test:.4f}")
#         print(f"Test Precision: {precision_test:.4f}")
#         print("-" * 40)
    
#     # Save results to file if requested
#     if save_to_file:
#         with open(filename, 'w') as f:
#             for res in results:
#                 f.write(f"Model: {res['model']}\n")
#                 f.write(f"CV F1 Score: {res['f1_mean']:.4f} (±{res['f1_std']:.4f})\n")
#                 f.write(f"CV Recall: {res['recall_mean']:.4f} (±{res['recall_std']:.4f})\n")
#                 f.write(f"CV Precision: {res['precision_mean']:.4f} (±{res['precision_std']:.4f})\n")
#                 f.write(f"Test F1 Score: {res['f1_test']:.4f}\n")
#                 f.write(f"Test Recall: {res['recall_test']:.4f}\n")
#                 f.write(f"Test Precision: {res['precision_test']:.4f}\n")
#                 f.write("-" * 40 + "\n")
#         print(f"Results saved to {filename}")

#     return results

# # Example usage:
# # from sklearn.ensemble import RandomForestClassifier
# # from sklearn.linear_model import LogisticRegression
# # models = [RandomForestClassifier(), LogisticRegression()]
# # get_score(models, X_train, X_test, y_train, y_test, cv=5, save_to_file=True)